In [1]:
import pandas as pd
df = pd.read_pickle('tweets_sentiment.pkl')

In [2]:
df['Location'].value_counts()

Washington, DC               7448
London                       5529
New York, NY                 5112
Ukraine                      4232
United States                3692
                             ... 
Frankfurt/Main                  1
vihiga Kenya                    1
lymm                            1
Global🌎 🚫No lists               1
Ontario via Massachusetts       1
Name: Location, Length: 15907, dtype: int64

In [3]:
from geopy.geocoders import Nominatim

geoloc = Nominatim(user_agent='ADG_NLP_Project')

In [4]:
from ratelimiter import RateLimiter

rate_limiter = RateLimiter(max_calls=1, period=1)

In [5]:
geo_cache = dict()

In [28]:
vc = df['Location'].value_counts()
vc2 = vc[vc > 10]
for loc, count in tqdm(vc2.iteritems(), total=len(vc2)):
    if loc not in geo_cache:
        with rate_limiter:
            try:
                geo_cache[loc] = geoloc.geocode(loc)
            except:
                pass

100%|██████████| 1476/1476 [29:30<00:00,  1.20s/it]


In [30]:
from tqdm import tqdm

data = []
for i, row in tqdm(df.iterrows(), total=len(df)):
    loc = row['Location']
    if loc in geo_cache and geo_cache[loc] is not None:
        df.loc[i, 'Latitude'] = geo_cache[loc].latitude
        df.loc[i, 'Longitude'] = geo_cache[loc].longitude

100%|██████████| 197276/197276 [00:13<00:00, 14992.68it/s]


In [11]:
df.to_pickle('tweets_sentiment_location.pkl')